<a href="https://colab.research.google.com/github/jamesETsmith/2022_simons_collab_pyscf_workshop/blob/main/demos/05_Excited_States.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the Jupyter notebook

* We need to install a few things before we get started
  * [PySCF](https://pyscf.org/) (for the quantum chemsitry)
  * [py3DMol](https://3dmol.csb.pitt.edu/) for visualizing the molecule
  * [plotly](https://plotly.com/python/) and kaleido for plotting

# Chemical System Setup

In this example, we're going to study the electronic excitations of 1,2-oxazole with [time-dependent density functional theory (TD-DFT)](https://en.wikipedia.org/wiki/Time-dependent_density_functional_theory#:~:text=Time%2Ddependent%20density%2Dfunctional%20theory,as%20electric%20or%20magnetic%20fields.).
We point out that just like the other examples, we cut corners (e.g. using a small basis set) to make these calculations run quicker on Google Colab.
For research, more careful choices of basis set, functional, etc are required for accurate results.
To learn more about running TD-DFT in PySCF, see the [user guide](https://pyscf.org/user/tddft.html).

In [1]:
from pyscf import gto, scf, dft, tddft
import py3Dmol

In [2]:
mol = gto.M(atom="""  C      1.0701      0.4341     -0.0336
  C      0.8115     -0.9049     -0.1725
  C     -0.6249     -1.0279     -0.0726
  H      1.9842      1.0231     -0.0364
  H      1.5156     -1.7176     -0.3255
  H     -1.2289     -1.9326     -0.1322
  O     -0.0873      1.1351      0.1422
  N     -1.1414      0.1776      0.1122""",
  # basis = "631g", 
  verbose=3)

# mol = gto.M('''
# C       -0.549658962      0.000000000     -5.749340540
# C       -0.549658962     -0.661620007     -4.508549950
# C       -0.549658962      0.118696949     -3.317807861
# C       -0.549658962      1.517490905     -3.339171452
# C       -0.549658962      2.139727367     -4.581749938
# C       -0.549658962      1.388227459     -5.775117065
# C       -0.549658962     -2.040455663     -4.103365626
# C       -0.549658962     -2.058324587     -2.734136993
# N       -0.549658962     -0.761110969     -2.254104063
# H       -0.549658962     -0.569660768     -6.674502447
# H       -0.549658962      2.097112517     -2.420198888
# H       -0.549658962      3.224412090     -4.634500730
# H       -0.549658962      1.908128210     -6.728608178
# H       -0.549658962     -2.907204188     -4.748630331
# H       -0.549658962     -2.894567717     -2.049797539
# H       -0.549658962     -0.500851598     -1.281922210''', verbose=3)

xyz_view = py3Dmol.view(width=400,height=400)
xyz_view.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view.setBackgroundColor('0xeeeeee')
xyz_view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# Helper Functions

Here we create helper functions to run DFT, TD-DFT, and perform the spectral analysis. Note that a gaussian distribution is a rough way to approximate the line shape for each excitation.

In [5]:
import numpy as np
from scipy.constants import physical_constants

ha_2_ev = 1/physical_constants["electron volt-hartree relationship"][0]

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def run_spectral_analysis(mol, xc="lda"):
    n_states=5
    spectral_width=0.1

    # Ground State DFT
    mf = dft.RKS(mol, xc=xc).run()

    # Excited State DFT
    mytd = tddft.TDDFT(mf)
    mytd.nstates = n_states
    mytd.max_space = 100
    mytd.max_cycle = 200
    mytd.kernel();
    mytd.analyze()
    osc_strengths = mytd.oscillator_strength()[:n_states-5]

    # Convolve lineshapes to make spectra
    energies_ev = mytd.e[:n_states]*ha_2_ev
    x_range = np.linspace(energies_ev.min()*0.9, energies_ev.max()*1.1, num=1000)
    intensity = np.zeros(x_range.size)

    for e, f in zip(energies_ev, osc_strengths):
        intensity += gaussian(x_range, e, spectral_width) * f

    # Rough Normalization
    dx = (x_range[-1] - x_range[0])/x_range.size
    area = (intensity*dx).sum()
    intensity /= area


    return x_range, intensity

In [6]:
import time
import pandas as pd

data = {"Excitation Energy (eV)":[], "Intensity":[], "Exchange-Correlation Functional":[]}

xcs = ["lda", "pbe", 'b3lyp']

for xc in xcs:
    ti = time.time()
    x_range, intensity = run_spectral_analysis(mol, xc=xc)

    data["Excitation Energy (eV)"] += x_range.tolist()
    data["Intensity"] += intensity.tolist()
    data["Exchange-Correlation Functional"] += [xc]*x_range.size
    tf = time.time()
    print(f"Time for {xc.upper()} calculations: {tf-ti:.2f}\n")

df = pd.DataFrame(data)


converged SCF energy = -238.456268099991
Excited State energies (eV)
[5.85859007 7.17536117 7.5577285  7.95623443 8.5217847 ]

** Singlet excitation energies and oscillator strengths **
Excited State   1:      5.85859 eV    211.63 nm  f=0.0064
Excited State   2:      7.17536 eV    172.79 nm  f=0.0356
Excited State   3:      7.55773 eV    164.05 nm  f=0.0090
Excited State   4:      7.95623 eV    155.83 nm  f=0.0367
Excited State   5:      8.52178 eV    145.49 nm  f=0.0043
Time for LDA calculations: 18.16



/tmp/ipykernel_181684/299890944.py:36: RuntimeWarning: invalid value encountered in divide
  intensity /= area


converged SCF energy = -242.610464570453
Excited State energies (eV)
[5.99520607 7.08325568 7.71895768 7.8787335  8.57186879]

** Singlet excitation energies and oscillator strengths **
Excited State   1:      5.99521 eV    206.81 nm  f=0.0068
Excited State   2:      7.08326 eV    175.04 nm  f=0.0374
Excited State   3:      7.71896 eV    160.62 nm  f=0.0084
Excited State   4:      7.87873 eV    157.37 nm  f=0.0355
Excited State   5:      8.57187 eV    144.64 nm  f=0.0043
Time for PBE calculations: 30.57



/home/daniil_artamonov/miniconda3/envs/sandbox/lib/python3.11/site-packages/pyscf/dft/libxc.py:1102: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


converged SCF energy = -242.868559547685
TD-SCF states [0, 1, 2, 3, 4] not converged.
Excited State energies (eV)
[0.4186441  2.40246079 6.25681158 7.1804755  7.40428159]

** Singlet excitation energies and oscillator strengths **
Excited State   1:      0.41864 eV   2961.57 nm  f=nan
Excited State   2:      2.40246 eV    516.07 nm  f=0.2909
Excited State   3:      6.25681 eV    198.16 nm  f=0.0108
Excited State   4:      7.18048 eV    172.67 nm  f=0.0424
Excited State   5:      7.40428 eV    167.45 nm  f=0.4357
Time for B3LYP calculations: 395.85



/home/daniil_artamonov/miniconda3/envs/sandbox/lib/python3.11/site-packages/pyscf/tdscf/rhf.py:1031: RuntimeWarning: invalid value encountered in sqrt
  norm = numpy.sqrt(.5/norm)  # normalize to 0.5 for alpha spin


# Plot the UV/Vis Spectra

Since we've stored our data in a dataframe, we can plot it quickly using `plotly.express`.

In [ ]:
import plotly.express as px
fig = px.line(df, x="Excitation Energy (eV)", y="Intensity", markers=True, color="Exchange-Correlation Functional")
fig.show()

In [17]:
df

,Excitation Energy (eV),Intensity,Exchange-Correlation Functional
0,9.536734,1.415932e-27,lda
1,10.085746,7.432869e-09,lda
2,10.634759,3.169599e-03,lda
3,11.183771,1.097959e-10,lda
4,11.732783,3.589503e-31,lda
...,...,...,...
2995,568.827948,0.000000e+00,wB97X_V
2996,569.389544,0.000000e+00,wB97X_V
2997,569.951141,0.000000e+00,wB97X_V
2998,570.512738,0.000000e+00,wB97X_V
